![imagen.png](attachment:imagen.png)

### Roberto Pacho


### Objetivo:

*  Consolidar los conocimientos adquiridos en clase para desarrollar simulaciones basados en eventos discretos.

### Introducción:

El golpe económico de la crisis sanitaria del corona virus no va a ser cosa de semanas, sino de meses. Dentro de una de las etapas importantes posteriores a las elecciones presidenciales son la re activación de los pequeños comercios y/o centros educativos. Para ello se plantea realizar  un modelo de simulación de eventos discretos que permita determinar cual son los principales factores para la re activación de la educación en el ingreso a las aulas de los estudiantes[1].

### Enunciado

* Diseñe y desarrolle un modelo y/o script que permita simular el siguiente caso real:

En base a los datos del siguiente link https://educacion.gob.ec/wp-content/uploads/downloads/2012/08/AZUAY11.pdf, genere una simulación del ingresos de los estudiantes, para ello debemos escoger un establecimiento y en base a los docentes y estudiantes modelar el reingreso de los estudiantes en base a los siguientes datos.
* Solo se va a tener en cuenta uno de los planteles educativos(Escuela, colegio, universidad dentro del Azuay).
* Se tiene un promedio que el 90% de los docentes han sido vacunados y pueden realizar el proceso de ingreso en cada uno de los cursos.
* Dentro del procesos se tiene que alrededor del 5% - 10% de los estudiantes no podrán asistir debido a no presentar la vacuna/enfermedades adyacentes.
* Los estudiantes solo pertenecen a una sola entidad educativa al igual que los docentes.
* Se va  a tener un periodo de prueba de un mes, posterior a ello se realiza al azar al 10% de estudiantes una prueba PCR para validar que no estén contagiados.
        * De la ultima el 2% de los estudiantes dan positivo por lo que se cierra el curso completo.
        * Los estudiantes asisten cada semana y estos están en un horario de 6 horas ya sea diurno o nocturno.
        * Tienen un receso 30 minutos dentro del establecimiento en donde se concentran todos los estudiantes y exite un foco de contagio del 2%.
* El proceso de simulación desarrollado deberá considerar los siguientes aspectos:
        * Generar un cuaderno de Python para el desarrollo y parametrización de graficas, reportes, y animación (Simpy).
        * Generar una animación en 2D/3D del modelo propuesto.
        * Obtener los siguientes análisis:
                * Cuantos contagiados tengo al final del mes.
                * Cuantos cursos debo cerrar.
                * Cuantos estudiantes y docentes ingresan y salen al final del mes.
            * Opinión
            * Conclusiones
            * Recomendaciones.
    * Fechas de Presentación: 04/07/2021 – 23:55 Subir al Avac en formato PDF y al Git los cuadernos de Python desarrollados.

Desarrollo del modelo de simulación

    Número de docentes: 60
    Número de cursos: 22
    Número de estudiantes: 745

Con estas características procedemos a inicializar los parámetros necesarios para realizar nuestra simulación:



## Parámetros iniciales de la simulación.

Tomando en cuenta los requerimientos se obtienen lso siguientes parámetros:

* Existira un total de 50 docentes ya que el 90% debe asistir al plantel.
* Existira un total de 735 estudiantes que asistirán al plantel educativo ya que el 5%-10% no podra asistir por enfermedad o vacuna.
* Existira un horario de 6 horas, es decir 360 min para realizar la simulación de cada día.
* se tiene un receso de 30 min cada día en donde existira una posibilidad de contagio.

In [1]:
NUMERO_DOCENTES              = 50
NUMERO_ESTUDIANTES           = 735
CURSOS                       = 22
NUMERO_ESTUDIANTES_POR_CURSO = NUMERO_ESTUDIANTES//CURSOS
TIEMPO_SIMULACION            = 360*NUMERO_ESTUDIANTES
DIAS                         = 0
MES                          = 'SEPTIEMBRE'
RESCESO                      = 30

## Creamos los estudiantes que se van a clases  y maestros

Mediante la clase Student vamos a crear los estudiantes que van a ingresar en nuestra simulación. Para ello se crea una lista de estudiantes que contiene la clase estudiante.

In [2]:
class Estudiante:  
    def __init__(self,nombre,curso,infectado,examen):
        self.nombre    = nombre
        self.curso     = curso
        self.infectado = infectado
        self.examen    = examen
        
class Maestro:
    def __init__(self,nombre,infectado,examen):
        self.nombre      = nombre
        self.curso       = 'indefinido'
        self.institucion = 'Unidad Educativa Humboldt'
        self.infectado   = infectado
        self.examen      = examen
    
def llenarEstudiantes():
    count=1
    lista_estudiantes = {}
    cursos_nivel      = ['octavo','noveno','decimo','primero','segundo','tercero']
    cursos_id         = ['A','B','C','D']
    for i in cursos_nivel:
        for j in cursos_id:
            for z in range(0,NUMERO_ESTUDIANTES_POR_CURSO+3 if i=='tercero' else NUMERO_ESTUDIANTES_POR_CURSO):
                nombre='estudiante_{}'.format(count)
                estudiante = Estudiante(nombre,'{}_{}'.format(i,j),0,0)
                lista_estudiantes[nombre]=estudiante
                count+=1
    return lista_estudiantes

def llenarMaestros():
    count=1
    lista_maestros = {}
    for i in range(0,NUMERO_DOCENTES):
        nombre = 'maestro_{}'.format(count)
        maestro = Maestro(nombre,0,0)
        lista_maestros[nombre]=maestro
        count+=1
    return lista_maestros

Llenamos estudiantes que van a estar en nuestra simulación y procedemos inicializarlos. A continuación se ve como se crearón los estudiantes.

In [3]:
LISTA_ESTUDIANTES = llenarEstudiantes()
LISTA_MAESTROS    = llenarMaestros()

Una vez realizado esto unificamos los datos para tener un total de personas que iran al plantel y poder realizar toda la simulación.

In [4]:
LISTA_PERSONAS = {**LISTA_ESTUDIANTES,**LISTA_MAESTROS}

## Ejecución de la simulación

### Importamos librerias

Las librerías que utilizamos son ```simpy``` para nuestra simulación y ```random``` para la generación de valores randómicos.

In [9]:
import simpy
import random


class Colegio():
    def __init__(self,environment):
        self.env          = environment
        self.pcr          = simpy.Resource(self.env,5)
        
    def proceso_clases(self):
        yield self.env.timeout(165)
    
    def proceso_receso(self):
        global RESCESO
        yield self.env.timeout(RESCESO)
        
    def proceso_prueba(self):
        yield self.env.timeout(2)
        
    def proceso_dia_clases(self,persona):
        yield self.env.process(self.proceso_clases())
        yield self.env.process(self.proceso_receso())
        if(random.randint(1,100)>98):
            persona.infectado=1
            LISTA_PERSONAS[persona.nombre] = persona
        yield self.env.process(self.proceso_clases())
    
    def proceso_pcr(self,persona):
        with self.pcr.request() as pcr:
            yield pcr
            yield self.env.process(self.proceso_prueba())
            if(random.randint(1,100)>(95 if persona.infectado==1 else 98)):
                persona.infectado=2
                LISTA_PERSONAS[persona.nombre] = persona

class Simulacion():
    
    def ejecutar_simulacion(self,env,filtrado):
        colegio = Colegio(env)
        global DIAS
        if DIAS < 31:
            print('inrgresan {} personas a la instituación'.format(len(filtrado)))
            for i in filtrado.items():
                yield env.process(colegio.proceso_dia_clases(i[1]))
        else:
            print('se tomo una muestra de: {} para prueba PCR'.format(int(len(LISTA_PERSONAS)*0.1)))
            for i in random.choices(list(LISTA_PERSONAS.keys()),k=int(len(LISTA_PERSONAS)*0.1)):
                env.process(colegio.proceso_pcr(LISTA_PERSONAS[i]))
                
def filtrar_posibles_contagios():
    filtrado={}
    for i in LISTA_PERSONAS.items():
        if i[1].infectado!=1:
            filtrado[i[0]]=i[1]
    return filtrado

In [10]:
DIAS=0
while (DIAS<=31):
    print("="*10,'DÍA {}'.format(DIAS),"="*10)
    env = simpy.Environment()
    simulacion = Simulacion()
    env.process(simulacion.ejecutar_simulacion(env,filtrar_posibles_contagios()))
    env.run(until=TIEMPO_SIMULACION)
    if(DIAS==31):
        print('infectados confirmados: {}'.format(len([i for i in LISTA_PERSONAS.items() if i[1].infectado ==2]))) 
    else:
        print('posibles infectados con corte en el día: {}'.format(len([i for i in LISTA_PERSONAS.items() if i[1].infectado ==1])))
    DIAS+=1

========== DÍA 0 ==========
inrgresan 854 personas a la instituación
posibles infectados con corte en el día: 15
========== DÍA 1 ==========
inrgresan 839 personas a la instituación
posibles infectados con corte en el día: 33
========== DÍA 2 ==========
inrgresan 821 personas a la instituación
posibles infectados con corte en el día: 44
========== DÍA 3 ==========
inrgresan 810 personas a la instituación
posibles infectados con corte en el día: 62
========== DÍA 4 ==========
inrgresan 792 personas a la instituación
posibles infectados con corte en el día: 75
========== DÍA 5 ==========
inrgresan 779 personas a la instituación
posibles infectados con corte en el día: 88
========== DÍA 6 ==========
inrgresan 766 personas a la instituación
posibles infectados con corte en el día: 97
========== DÍA 7 ==========
inrgresan 757 personas a la instituación
posibles infectados con corte en el día: 115
========== DÍA 8 ==========
inrgresan 739 personas a la instituación
posibles infectados con co

# Reportes
Resumenes de la información de la simulación y analizar resultados

In [11]:
import altair as alt
import pandas as pd
import plotly.express as px
import plotly
plotly.offline.init_notebook_mode(connected=True)

## Estadísticas generales

In [12]:
print('Número total de personas en la institución: {}'.format(len(LISTA_PERSONAS)))
print('Número total de estudiantes: {}'.format(len(LISTA_ESTUDIANTES)))
print('Número total de maestros: {}'.format(len(LISTA_MAESTROS)))
print('Número total de personas con posible contagio: {}'.format(len([i for i in LISTA_PERSONAS.items() if i[1].infectado ==1])))
print('Número total de personas contagiadas confirmadas: {}'.format(len([i for i in LISTA_PERSONAS.items() if i[1].infectado ==2])))
print('Cursos que se cierran: {}'.format(list(pd.DataFrame([i[1].curso for i in LISTA_PERSONAS.items() if i[1].infectado ==2],columns=['cursos'])['cursos'].unique())))

Número total de personas en la institución: 854
Número total de estudiantes: 804
Número total de maestros: 50
Número total de personas con posible contagio: 386
Número total de personas contagiadas confirmadas: 4
Cursos que se cierran: ['octavo_B', 'segundo_C', 'segundo_D', 'tercero_B']


## Estudiantes y docentes que entran y salen al fin del mes

In [13]:
filtrado_estudiantes = {}
filtrado_maestros    = {}
for i in LISTA_PERSONAS.items():
    if(i[0].find('maestro')!=-1):
        filtrado_maestros[i[0]]=i[1]
    else:
        filtrado_estudiantes[i[0]]=i[1]

no_infectados = 0
infectados    = 0
po_infectados = 0
for i in filtrado_maestros.items():
    if i[1].infectado==1:
        po_infectados+=1
    elif i[1].infectado==2:
        infectados+=1
    else:
        no_infectados+=1
df_maestro = pd.DataFrame([('no infectados',no_infectados),('posibles infectados',po_infectados),
                          ('infectados',infectados)],columns=['description','value'])
df_maestro

,description,value
0,no infectados,33
1,posibles infectados,17
2,infectados,0


In [14]:
no_infectados = 0
infectados    = 0
po_infectados = 0
for i in filtrado_estudiantes.items():
    if i[1].infectado==1:
        po_infectados+=1
    elif i[1].infectado==2:
        infectados+=1
    else:
        no_infectados+=1
df_estudiantes = pd.DataFrame([('no infectados',no_infectados),('posibles infectados',po_infectados),
                          ('infectados',infectados)],columns=['description','value'])
df_estudiantes

,description,value
0,no infectados,431
1,posibles infectados,369
2,infectados,4


In [15]:
alt.Chart(df_estudiantes).mark_bar().encode(
    y=alt.Y('description',sort=alt.EncodingSortField(field="value",order='ascending'),title='descripción'),
    x=alt.X('value',title='cantidad'),
    color= alt.condition(
            alt.datum.value == max(df_estudiantes['value']),
            alt.value('red'),    
            alt.value('lightgrey')
            )
).properties(
    title='Estudiantes contagiados/ no contagiados / posibles contagiados'
)&alt.Chart(df_maestro).mark_bar().encode(
    y=alt.Y('description',sort=alt.EncodingSortField(field="value",order='ascending'),title='descripción'),
    x=alt.X('value',title='cantidad'),
    color= alt.condition(
            alt.datum.value == max(df_maestro['value']),
            alt.value('red'),    
            alt.value('lightgrey')
            )
).properties(title='Maestros contagiados/ no contagiados / posibles contagiados')

alt.VConcatChart(...)

In [16]:
print('Al final del mes salen un total de : {} maestros'.format(df_maestro[1:3]['value'].sum()))
print('Al final del mes salen un total de : {} estudiantes'.format(df_estudiantes[1:3]['value'].sum()))

Al final del mes salen un total de : 17 maestros
Al final del mes salen un total de : 373 estudiantes


### Porcentaje de contagiados / no contagiados / posibles contagiados en la institución

In [19]:
no_infectados = 0
infectados    = 0
po_infectados = 0
for i in LISTA_PERSONAS.items():
    if i[1].infectado==1:
        po_infectados+=1
    elif i[1].infectado==2:
        infectados+=1
    else:
        no_infectados+=1

df_general = pd.DataFrame([('no infectados',no_infectados/len(LISTA_PERSONAS)),
                           ('posibles infectados',po_infectados/len(LISTA_PERSONAS)),
                          ('infectados',infectados/len(LISTA_PERSONAS))],columns=['description','value'])
df_general

,description,value
0,no infectados,0.543326
1,posibles infectados,0.451991
2,infectados,0.004684


In [20]:
fig = px.pie(df_general, values="value",
             names="description", 
             color_discrete_sequence=['red','lightgrey','darkgrey'],width=1000,height=500)
fig.update_layout(title_text='contagiados / no contagiados / posibles contagiados en la institución de la Salle', title_x=0.5)